In [ ]:
from dotenv import load_dotenv
import os
from hera.workflows import EmptyDirVolume, models, script, Artifact, Parameter, DAG, Steps, Step, NoneArchiveStrategy, Workflow
from hera.shared import global_config

load_dotenv("/home/otto/s1_zarr/.env")

In [ ]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "inca"
global_config.token = os.getenv("argo_token")
global_config.image = "ghcr.io/oscipal/image_zarr:latest"

In [ ]:
empty_dir = [models.Volume(name="emptydir", empty_dir=models.EmptyDirVolumeSource())]

In [ ]:
@script(volume_mounts=[models.VolumeMount(name="emptydir", mount_path="/mnt/vol/")])
def download_and_use():
    from urllib.request import urlretrieve
    import datetime
    import os
    import xarray as xr

    mount_path = "/mnt/vol/"

    url = f"https://public.hub.geosphere.at/datahub/resources/inca-v1-1h-1km/filelisting/RR/INCAL_HOURLY_RR_202506.nc"
    urlretrieve(url, os.path.join(mount_path, "INCA.nc"))

    ds = xr.open_dataset(os.path.join(mount_path, "INCA.nc")).isel(time=1, x=1, y=1)["RR"].load()
    print(ds.values)

In [ ]:
with Workflow(
    generate_name="empty-dir-",
    volumes = empty_dir,
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        download_and_use()

In [ ]:
w.create()